In [1]:
import gradio as gr
import sqlite3
from datetime import datetime

# ---------- DATABASE SETUP ----------
def init_db():
    conn = sqlite3.connect("blog.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS posts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            content TEXT NOT NULL,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

init_db()   # create db if not exists


# ---------- DATABASE OPERATIONS ----------

def add_post(title, content):
    if title.strip() == "" or content.strip() == "":
        return "Title and content cannot be empty."

    conn = sqlite3.connect("blog.db")
    cursor = conn.cursor()

    cursor.execute("INSERT INTO posts (title, content, created_at) VALUES (?, ?, ?)",
                   (title, content, datetime.now().strftime("%Y-%m-%d %H:%M")))
    conn.commit()
    conn.close()

    return "Post added successfully!"


def view_posts():
    conn = sqlite3.connect("blog.db")
    cursor = conn.cursor()

    cursor.execute("SELECT title, content, created_at FROM posts ORDER BY id DESC")
    posts = cursor.fetchall()
    conn.close()

    if not posts:
        return "No posts found."

    output = ""
    for p in posts:
        title, content, date = p
        output += f"### {title}\n📅 *{date}*\n\n{content}\n\n---\n"

    return output


# ---------- GRADIO UI ----------

with gr.Blocks() as app:
    gr.Markdown("# 📝 Simple Blogging Website (Python + SQLite + Gradio)")

    with gr.Tab("Create Post"):
        title_input = gr.Textbox(label="Title")
        content_input = gr.Textbox(label="Content", lines=5)
        output_msg = gr.Textbox(label="Status")

        submit_btn = gr.Button("Add Post")
        submit_btn.click(add_post, [title_input, content_input], output_msg)

    with gr.Tab("View Posts"):
        show_btn = gr.Button("Load All Posts")
        posts_display = gr.Markdown()

        show_btn.click(view_posts, None, posts_display)

app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
